<a href="https://colab.research.google.com/github/namanko/summer-of-code-2024/blob/main/week1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import userdata
import os

In [2]:
os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')
os.environ['KAGGLE_USER'] = userdata.get('KAGGLE_USER')


**Loading dataset**

In [3]:
!kaggle datasets download -d jainilcoder/online-payment-fraud-detection

Dataset URL: https://www.kaggle.com/datasets/jainilcoder/online-payment-fraud-detection
License(s): CC0-1.0
 98% 174M/178M [00:05<00:00, 34.1MB/s]
100% 178M/178M [00:05<00:00, 33.8MB/s]


In [4]:
!unzip online-payment-fraud-detection.zip

Archive:  online-payment-fraud-detection.zip
  inflating: onlinefraud.csv         


In [5]:
import pandas as pd
import numpy as np

In [6]:
fraud_csv = pd.read_csv('onlinefraud.csv')
fraud_csv

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.00,C776919290,0.00,339682.13,1,0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1,0
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.00,C2080388513,0.00,0.00,1,0


**Data Cleaning and Preprocressing**

In [7]:
train_csv =  fraud_csv.drop(['isFlaggedFraud'],axis=1)
train_csv

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0
...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.00,C776919290,0.00,339682.13,1
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.00,C2080388513,0.00,0.00,1


In [8]:
train_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 10 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
dtypes: float64(5), int64(2), object(3)
memory usage: 485.4+ MB


In [9]:
train_csv = train_csv.dropna()


In [10]:
train_csv.duplicated().sum()

0

In [11]:
train_csv['type'].unique()

array(['PAYMENT', 'TRANSFER', 'CASH_OUT', 'DEBIT', 'CASH_IN'],
      dtype=object)

In [12]:
train_csv_enc = pd.get_dummies(train_csv,columns=['type'])

In [13]:
train_csv_enc['type_CASH_IN'] = train_csv_enc['type_CASH_IN'].astype(int)
train_csv_enc['type_CASH_OUT'] = train_csv_enc['type_CASH_OUT'].astype(int)
train_csv_enc['type_DEBIT'] = train_csv_enc['type_DEBIT'].astype(int)
train_csv_enc['type_PAYMENT'] = train_csv_enc['type_PAYMENT'].astype(int)
train_csv_enc['type_TRANSFER'] = train_csv_enc['type_TRANSFER'].astype(int)

In [14]:
train_csv_enc

,step,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,1,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0,0,0,1,0
1,1,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0,0,0,1,0
2,1,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0,0,0,0,1
3,1,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0,1,0,0,0
4,1,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,339682.13,C786484425,339682.13,0.00,C776919290,0.00,339682.13,1,0,1,0,0,0
6362616,743,6311409.28,C1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1,0,0,0,0,1
6362617,743,6311409.28,C1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1,0,1,0,0,0
6362618,743,850002.52,C1685995037,850002.52,0.00,C2080388513,0.00,0.00,1,0,0,0,0,1


**Feature Engineering**

In [15]:
train_csv_enc['Origdiff'] = train_csv_enc['newbalanceOrig'] - train_csv_enc['oldbalanceOrg']
train_csv_enc['Destdiff'] = train_csv_enc['newbalanceDest'] - train_csv_enc['oldbalanceDest']

In [16]:
train_csv_enc

,step,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,Origdiff,Destdiff
0,1,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0,0,0,1,0,-9839.64,0.00
1,1,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0,0,0,1,0,-1864.28,0.00
2,1,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0,0,0,0,1,-181.00,0.00
3,1,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0,1,0,0,0,-181.00,-21182.00
4,1,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0,0,0,1,0,-11668.14,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,339682.13,C786484425,339682.13,0.00,C776919290,0.00,339682.13,1,0,1,0,0,0,-339682.13,339682.13
6362616,743,6311409.28,C1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1,0,0,0,0,1,-6311409.28,0.00
6362617,743,6311409.28,C1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1,0,1,0,0,0,-6311409.28,6311409.27
6362618,743,850002.52,C1685995037,850002.52,0.00,C2080388513,0.00,0.00,1,0,0,0,0,1,-850002.52,0.00


**Dealing with Class Imbalance - SMOTE**

In [17]:
x_train = train_csv_enc.drop(['step','amount','nameOrig','oldbalanceOrg','newbalanceOrig','nameDest','oldbalanceDest','newbalanceDest','isFraud'],axis=1)
y_train = train_csv_enc['isFraud']

In [18]:
x_train

,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,Origdiff,Destdiff
0,0,0,0,1,0,-9839.64,0.00
1,0,0,0,1,0,-1864.28,0.00
2,0,0,0,0,1,-181.00,0.00
3,0,1,0,0,0,-181.00,-21182.00
4,0,0,0,1,0,-11668.14,0.00
...,...,...,...,...,...,...,...
6362615,0,1,0,0,0,-339682.13,339682.13
6362616,0,0,0,0,1,-6311409.28,0.00
6362617,0,1,0,0,0,-6311409.28,6311409.27
6362618,0,0,0,0,1,-850002.52,0.00


In [19]:
y_train

0          0
1          0
2          1
3          1
4          0
          ..
6362615    1
6362616    1
6362617    1
6362618    1
6362619    1
Name: isFraud, Length: 6362620, dtype: int64

In [20]:
x = np.array(x_train)
y = np.array(y_train)


In [34]:
from sklearn.model_selection import train_test_split
X_tr,X_te,Y_tr,Y_te = train_test_split(x,y,test_size=0.25,random_state=42)

In [35]:
print("Before OverSampling, counts of label '1': {}".format(sum(Y_tr == 1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(Y_tr == 0)))

Before OverSampling, counts of label '1': 6168
Before OverSampling, counts of label '0': 4765797 



In [36]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 2)
X,Y = sm.fit_resample(X_tr,Y_tr)

In [37]:
print("After OverSampling, counts of label '1': {}".format(sum(Y == 1)))
print("After OverSampling, counts of label '0': {}".format(sum(Y == 0)))

After OverSampling, counts of label '1': 4765797
After OverSampling, counts of label '0': 4765797


**Logistic Regression Model**

In [39]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X,Y)

LogisticRegression()

In [41]:
pred = lr.predict(X_te)

**Model Evaluation**

In [42]:
from sklearn.metrics import classification_report, roc_auc_score
print(classification_report(Y_te,pred))
print('Roc-auc-score:',roc_auc_score(Y_te,pred))

              precision    recall  f1-score   support

           0       1.00      0.70      0.83   1588610
           1       0.00      0.99      0.01      2045

    accuracy                           0.70   1590655
   macro avg       0.50      0.85      0.42   1590655
weighted avg       1.00      0.70      0.83   1590655

Roc-auc-score: 0.8470646179605985


**Random Forest Model**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from random import randint

rf = RandomForestClassifier()

rf.fit(X, Y)

In [ ]:
pred_new = rf.predict(X_te)

In [ ]:
print(classification_report(Y_te,pred_new))
print('Roc-auc-score:',roc_auc_score(Y_te,pred_new))